# Lectura de datos

In [65]:
#Paquetes para lectura y recuento
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

#Primero importamos los datos que vamos a leer
news = fetch_20newsgroups()

#Pasamos los datos a un DataFrame (datos en pandas)
data = pd.DataFrame(news['data'], columns = ['News'])
target = pd.DataFrame(news['target'], columns = ['Type'])

In [122]:
data#['News'][0]

In [123]:
target

# Extacción de características (Siempre antes de sobremuestrear)

In [106]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Inicialización de los vectores de extracción de características
#CountVectorizer
vectCV = CountVectorizer(max_features = 400).fit(data['News']) #stop_words = ['palabras', 'que', 'no', 'cuentan']
#TfidfVectorizer. 
vectTf = TfidfVectorizer(min_df = 3,use_idf = False, smooth_idf = False).fit(data['News']) 

#Obtención de las matrices de características
X_vectorizedCV = vectCV.transform(data['News'])
X_vectorizedTf = vectTf.transform(data['News'])

In [125]:
X_vectorizedCV
#print(X_vectorizedCV.todense())

In [126]:
X_vectorizedTf
#print(X_vectorizedTf.todense())

# Sobremuestreo y entrenamiento del modelo

## Sobremuestreo 

In [127]:
#Importamos los paquetes de sobremuestreo
from imblearn.over_sampling import SMOTE

#SMOTE
smote = SMOTE()

#Generación de nuevas muestras sintéticas
dataSmoteCV, targetSmoteCV = smote.fit_resample(X_vectorizedCV,target['Type'])
dataSmoteTf, targetSmoteTf = smote.fit_resample(X_vectorizedTf,target['Type'])
print(dataSmoteCV)

## Obtención de los conjuntos de evaluación y entrenamiento

In [116]:
from sklearn.model_selection import train_test_split
dataTrainCV, dataTestCV, targetTrainCV, targetTestCV = train_test_split(dataSmoteCV,targetSmoteCV, random_state = 0)
dataTrainTf, dataTestTf, targetTrainTf, targetTestTf = train_test_split(dataSmoteTf,targetSmoteTf, random_state = 0)

## Árbol de decisión

In [128]:
#Paquete
from sklearn.tree import DecisionTreeClassifier

#Inicialización.
modelDTSCV = DecisionTreeClassifier()
modelDTSTf = DecisionTreeClassifier()

#Entrenamiento
modelDTSCV.fit(dataTrainCV, targetTrainCV)
modelDTSTf.fit(dataTrainTf, targetTrainTf)

#Predicción 
targetPredDTSCV = modelDTSCV.predict(dataTestCV)
targetPredDTSTf = modelDTSTf.predict(dataTestTf)

#Evaluación

#Paquete
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print('Exactitud árbol CV: ', accuracy_score(targetPredDTSCV,targetTestCV))
print('Exactitud árbol Tfidf: ', accuracy_score(targetPredDTSTf,targetTestTf))
print('Memoria árbol CV: ', recall_score(targetPredDTSCV,targetTestCV,average='weighted'))
print('Memoria árbol Tfidf: ', recall_score(targetPredDTSTf,targetTestTf,average='weighted'))
print('Precisión árbol CV: ', precision_score(targetPredDTSCV,targetTestCV,average='weighted'))
print('Precisión árbol Tfidf: ', precision_score(targetPredDTSTf,targetTestTf,average='weighted'))
print('Puntuación F1 árbol CV: ', f1_score(targetPredDTSCV,targetTestCV,average='weighted'))
print('Puntuación F1 árbol Tfidf: ', f1_score(targetPredDTSTf,targetTestTf,average='weighted'))